<a href="https://colab.research.google.com/github/rifqij/bigData/blob/main/project_ICW_1_PysparkEDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Check is Link to Drive is OK
google = !if [ -d 'Drive/' ]; then echo "1" ; else echo "0"; fi
if (google[0] is '0' ):
  from google.colab import drive
  drive.mount('/content/Drive/')
!if [ -d 'Drive/' ]; then echo "Connection to Google drive successful" ; else echo "Error to connect to Google drive"; fi\

Mounted at /content/Drive/
Connection to Google drive successful


In [2]:
# 1. Install all the dependencies in Colab environment i.e. Apache Spark 3.0.1 with hadoop 2.7, Java 8 and Findspark to locate the spark in the system
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop2.7.tgz
!tar xf spark-3.0.1-bin-hadoop2.7.tgz
!pip install -q findspark

In [3]:
# 2. Setup Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop2.7"

In [4]:
# 3. Start Spark Session
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [5]:
from pyspark.sql import sqlContext

dataframe_mysql = sqlContext.read.format("jdbc").options(
    url="jdbc:mysql://localhost:101.255.89.94/opentender3_201812",
    driver = "com.mysql.jdbc.Driver",
    dbtable = "ruppenyedia",
    user="reseach",
    password="aeThoG8eeSh6ohvu").load()

ImportError: ignored

In [6]:
# Import data
df = spark.read.csv("/content/Drive/MyDrive/project ICW/ruppenyedia.csv", sep=";", encoding="latin1", header=True)

In [7]:
df.show(10)

+--------------------+-------+------------+------------+--------------------+--------------------+------------------+------+-----------------+----------------+---------------+-----------------+-------------------+---------------+-----------------+----------+----------------+---------------+----------------+-------------------+--------------------+---------------------+--------------------+---------------------+--------------------+-------------------+--------------------+------------+--------------------+-------------+--------------------+------------------+--------------------+------------+
|                  id|  idrup|idrup_client|id_swakelola|           namapaket|          keterangan|       spesifikasi|lokasi|    tahunanggaran|idjenispengadaan| jenispengadaan|idmetodepengadaan|    metodepengadaan|status_penyedia|    statuspradipa|statustkdn|statususahakecil|  statusumumkan|statusaktifpaket|  statusdeletepaket|tanggalawalpemilihan|tanggalakhirpemilihan|tanggalawalpekerjaan|tanggalak

In [36]:
import pyspark.sql.functions as F

# indicator 25
# count character
df1 = df.withColumn("count_namapaket", F.length('namapaket'))

# create conditions and assign the category
'''
df['indikasi_namapaket'] = ''
df.loc[(df['judul_count'] < 10), 'idikasi_namapaket'] = 1
df.loc[(df['judul_count'] >= 10), 'idikasi_namapaket'] = 0
'''
df1 = df1.withColumn("indikasi_namapaket", F.when(F.length('namapaket') < 10,1).otherwise(0))

# sum the row with value in judul_indikasi
'''
judul_sum = df.['judul_indikasi'].sum()
judul_all = df.len()

print("count: "+str(df.select(count("indikasi")).collect()[0]))
'''

#find the percentage of judul with fewer than character
'''
percent_judul = (judul_sum*100)/judul_all
df1.show()
'''
print("Indicator24: Percent of tender with fewer than 10 characters in the title = " + str(df1.select(F.avg("indikasi_namapaket")*100).collect()[0][0]))

Indicator24: Percent of tender with fewer than 10 characters in the title = 6.683167533947295


In [37]:
# indicator 25
# Not considering the exact same value as the judul
df1 = df.withColumn("count_keterangan", F.length('keterangan'))
df1 = df1.withColumn("indikasi_keterangan", F.when(F.length('keterangan') < 30, 1).otherwise(0))
print("Indicator26: Percent of tender with fewer than 30 characters in the description = " + str(df1.select(F.avg("indikasi_keterangan")*100).collect()[0][0]))

Indicator26: Percent of tender with fewer than 30 characters in the description = 36.347693608111456


In [16]:
# indicator 26
import pyspark.sql.functions as F
# Percent of tenders that do not include detailed item codes or item descriptions [ONGOING]
#df1 = df1.withColumn('idjenispengadaan', F.regexp_replace('idjenispengadaan', '-', None))
#df1 = df1.withColumn('jenispengadaan', F.regexp_replace('jenispengadaan', '-', None))
df1 = df.withColumn("isnull_idjenispengadaan", F.when(df["idjenispengadaan"] == "-", 1).otherwise(0))
df1 = df1.withColumn("isnull_jenispengadaan", F.when(df["jenispengadaan"] == "-", 1).otherwise(0))
print("Indicator26: Percent of tenders that do not include detailed item codes or item descriptions = " + str(df1.select(F.avg("isnull_idjenispengadaan")*100).collect()[0][0]))
print("Indicator26: Percent of tenders that do not include detailed item codes or item descriptions = " + str(df1.select(F.avg("isnull_jenispengadaan")*100).collect()[0][0]))

Indicator26: Percent of tenders that do not include detailed item codes or item descriptions = 36.70177499306149
Indicator26: Percent of tenders that do not include detailed item codes or item descriptions = 36.70305990206226


In [15]:
df1.show()

+--------------------+-------+------------+------------+--------------------+--------------------+------------------+------+-----------------+----------------+---------------+-----------------+-------------------+---------------+-----------------+----------+----------------+---------------+----------------+-------------------+--------------------+---------------------+--------------------+---------------------+--------------------+-------------------+--------------------+------------+--------------------+-------------+--------------------+------------------+--------------------+------------+-----------------------+---------------------+
|                  id|  idrup|idrup_client|id_swakelola|           namapaket|          keterangan|       spesifikasi|lokasi|    tahunanggaran|idjenispengadaan| jenispengadaan|idmetodepengadaan|    metodepengadaan|status_penyedia|    statuspradipa|statustkdn|statususahakecil|  statusumumkan|statusaktifpaket|  statusdeletepaket|tanggalawalpemilihan|tangga

In [ ]:
#Data Transform 'Nilai' untuk bg Ammar
import numpy as np

nilai = df.filter(['nilai_kontrak','nilai_negosiasi','nilai_terkoreksi','nilai_penawaran'],axis=1)
nilai.replace('-', np.nan, inplace=True)
nilai.nilai_kontrak = nilai.nilai_kontrak.fillna(value=nilai.nilai_negosiasi)
nilai.nilai_kontrak = nilai.nilai_kontrak.fillna(value=nilai.nilai_terkoreksi)
nilai.nilai_kontrak = nilai.nilai_kontrak.fillna(value=nilai.nilai_penawaran)

nilai

,nilai_kontrak,nilai_negosiasi,nilai_terkoreksi,nilai_penawaran
0,178650000.00,NaN,NaN,178650000.00
1,178650000.00,NaN,NaN,178650000.00
2,178650000.00,NaN,NaN,178650000.00
3,178650000.00,NaN,NaN,178650000.00
4,178650000.00,NaN,NaN,178650000.00
...,...,...,...,...
1051698,5.49054e+08,NaN,5.49054e+08,5.49054e+08
1051699,2.6598e+08,NaN,2.6598e+08,2.6598e+08
1051700,3.234e+08,NaN,3.234e+08,3.234e+08
1051701,6.90098e+08,NaN,6.90098e+08,6.90098e+08


In [ ]:
df['nilai'] = nilai.nilai_kontrak
df

,id,kd_tender,kd_rup_paket,kd_paket,tahun_anggaran,kd_lpse,kd_satker,nama_satker,kd_klpd,nama_klpd,jenis_klpd,pagu,hps,nilai_penawaran,nilai_negosiasi,nilai_terkoreksi,nilai_kontrak,tgl_pengumuman_tender,tgl_penetapan_pemenang,kd_penyedia,nama_penyedia,npwp_penyedia,jum_peserta,jum_penawar,sum_peserta,sum_penawar,sum_monopoli,persen_hps,skor_hps,skor_nilai,skor_monopoli,skor_peserta,skor_penawar,skor_total,skor_100,tanggal_skor,nilai
0,1,9012,-,5001,2008,12,dinkesprov,Dinas Kesehatan Prov. Kalteng,D226,Provinsi Kalimantan Tengah,PROVINSI,1.800000e+08,1.800000e+08,178650000.00,-,-,-,2009-05-07 00:00:00,2009-06-23 23:59:00.0,107012,ELZA PUTRI SULUNG,01.922.239.7-711.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,178650000.00
1,2,9012,-,5001,2008,12,dinkesprov,Dinas Kesehatan Prov. Kalteng,D226,Provinsi Kalimantan Tengah,PROVINSI,2.108160e+08,2.055780e+08,178650000.00,-,-,-,2009-05-07 00:00:00,2009-06-23 23:59:00.0,107012,ELZA PUTRI SULUNG,01.922.239.7-711.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,178650000.00
2,3,9012,-,5001,2008,12,dinkesprov,Dinas Kesehatan Prov. Kalteng,D226,Provinsi Kalimantan Tengah,PROVINSI,6.600000e+08,6.600000e+08,178650000.00,-,-,-,2009-05-07 00:00:00,2009-06-23 23:59:00.0,107012,ELZA PUTRI SULUNG,01.922.239.7-711.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,178650000.00
3,4,9012,-,5001,2008,12,dinkesprov,Dinas Kesehatan Prov. Kalteng,D226,Provinsi Kalimantan Tengah,PROVINSI,6.750000e+08,6.426000e+08,178650000.00,-,-,-,2009-05-07 00:00:00,2009-06-23 23:59:00.0,107012,ELZA PUTRI SULUNG,01.922.239.7-711.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,178650000.00
4,5,9012,-,5001,2008,12,dinkesprov,Dinas Kesehatan Prov. Kalteng,D226,Provinsi Kalimantan Tengah,PROVINSI,1.390600e+09,1.390285e+09,178650000.00,-,-,-,2009-05-07 00:00:00,2009-06-23 23:59:00.0,107012,ELZA PUTRI SULUNG,01.922.239.7-711.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,178650000.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1051698,1051699,16332099,26513555,15560099,2020,99,4.04.01,BADAN PENDAPATAN DAERAH PROVINSI BANTEN,D43,Provinsi Banten,PROVINSI,5.794000e+08,5.507000e+08,5.49054e+08,-,5.49054e+08,-,2020-11-20 14:00:00,-,9514047,PT. AMTEK SOLUSINDO,66.209.473.9-044.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.49054e+08
1051699,1051700,16361099,26511351,15582099,2020,99,2.13.01,DINAS KEPEMUDAAN DAN OLAHRAGA,D43,Provinsi Banten,PROVINSI,2.850000e+08,2.817309e+08,2.6598e+08,-,2.6598e+08,-,2020-11-18 10:00:00,-,1706260,CV.ABIRAMA KARYA SEJAHTERA,83.193.627.3-429.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.6598e+08
1051700,1051701,16385099,26576233,15603099,2020,99,1.02.01.02,RUMAH SAKIT UMUM MALIMPING,D43,Provinsi Banten,PROVINSI,3.424400e+08,3.300000e+08,3.234e+08,-,3.234e+08,-,2020-11-23 10:00:00,-,316262,PT. DIVA MULYA PRATAMA,31.479.395.1-607.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.234e+08
1051701,1051702,16485099,26609516,15687099,2020,99,2.07.01,DINAS PEMBERDAYAAN MASYARAKAT DESA,D43,Provinsi Banten,PROVINSI,8.491500e+08,7.114475e+08,6.90098e+08,-,6.90098e+08,-,2020-11-24 10:00:00,-,890245,PILAR PRATAMA. CV,94.316.406.1-401.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.90098e+08


In [ ]:
df.to_csv(r'/content/tenderselesai_fixedNilai.csv', index=False)

In [ ]:
# download the file
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('tenderselesai_fixedNilai.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>